In [48]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import pickle

In [33]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\azizd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# Load dataset
df_true = pd.read_csv("archive2/True.csv")
df_fake = pd.read_csv("archive2/Fake.csv")

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017"
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017"
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017"


In [45]:
df_fake.head(10)

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [4]:
# Add labels (true=1, fake=0)
df_true["label"] = 1
df_fake["label"] = 0

# Concatenate true and fake df's

df = pd.concat([df_true, df_fake])
df

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


In [5]:
df.describe()

,label
count,44898.000000
mean,0.477015
std,0.499477
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44898 entries, 0 to 23480
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   label    44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.1+ MB


In [7]:
# remove unnecessary columns
df = df[["title", "text", "label"]]
df.head(10)

,title,text,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,1
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",1
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",1
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,1
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,1
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,1


In [37]:
# clean text 
stop_words = set(stopwords.words("english"))
wordnetlemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z\s]", " ", text)
    text = [wordnetlemmatizer.lemmatize(token) for token \
            in text.split() if token not in stop_words]
    return " ".join(text).strip()

In [38]:
# data preprocessing 
df.loc[:, "title"] = df["title"].apply(clean_text)
df.loc[:, "text"] = df["text"].apply(clean_text)
df.head(10)

,title,text,label
0,u budget fight loom republican flip fiscal script,washington reuters head conservative republica...,1
1,u military accept transgender recruit monday p...,washington reuters transgender people allowed ...,1
2,senior u republican senator let mr mueller job,washington reuters special counsel investigati...,1
3,fbi russia probe helped australian diplomat ti...,washington reuters trump campaign adviser geor...,1
4,trump want postal service charge much amazon s...,seattlewashington reuters president donald tru...,1
5,white house congress prepare talk spending imm...,west palm beach flawashington reuters white ho...,1
6,trump say russia probe fair timeline unclear nyt,west palm beach fla reuters president donald t...,1
7,factbox trump twitter dec approval rating amazon,following statement posted verified twitter ac...,1
8,trump twitter dec global warming,following statement posted verified twitter ac...,1
9,alabama official certify senatorelect jones to...,washington reuters alabama secretary state joh...,1


In [39]:
df["title"]

0        u budget fight loom republican flip fiscal script
1        u military accept transgender recruit monday p...
2           senior u republican senator let mr mueller job
3        fbi russia probe helped australian diplomat ti...
4        trump want postal service charge much amazon s...
                               ...                        
23476    mcpain john mccain furious iran treated u sail...
23477    justice yahoo settle email privacy classaction...
23478    sunnistan u allied safe zone plan take territo...
23479    blow million al jazeera america finally call q...
23480    u navy sailor held iranian military sign neoco...
Name: title, Length: 44266, dtype: object

In [40]:
# Make sure there are no empty strings
df["title"].apply(lambda x: len(x.strip())).value_counts()

title
54     1493
55     1458
57     1444
58     1403
56     1382
       ... 
159       1
237       1
166       1
180       1
162       1
Name: count, Length: 169, dtype: int64

In [57]:
#df = df[df["title"].str.strip() != ""]
#df = df[df["text"].str.strip() != ""]

In [42]:
# feature extrection (tf-idf)
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
title_vector = tfidf_vectorizer.fit_transform(df["title"])
text_vector = tfidf_vectorizer.fit_transform(df["text"])

In [56]:
title_vector.shape

(44266, 5000)

In [44]:
# split the data
X = text_vector
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, random_state=42)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (35412, 5000)
X_test shape: (8854, 5000)
y_train shape: (35412,)
y_test shape: (8854,)


In [49]:
# Multinomial naive bayes

nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

MultinomialNB()

In [52]:
file_name = "nb_model.sav"
pickle.dump(nb_model, open(file_name, "wb"))

In [53]:
y_pred = nb_model.predict(X_test)

In [54]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9307657555906935

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.94      0.93      4568
           1       0.93      0.92      0.93      4286

    accuracy                           0.93      8854
   macro avg       0.93      0.93      0.93      8854
weighted avg       0.93      0.93      0.93      8854


Confusion Matrix:
 [[4281  287]
 [ 326 3960]]
